Цель: Получить резюме кандидатов из компаний, в которых наблюдается массовый отток сотрудников.

Задача: Реализовать скрипт в Python, возвращающий список резюме сооискателей по принципу совпадения текущего/последнего места работы кандидатов.
Необходимо определить компании, у которых текущие/бывшие сотрудники сейчас размещают резюме. И вывести резюме в сортировке по компаниям, где таких кандидатов больше всего.
Скрипт должен поддерживать конфигурирование основных параметров поиска(ключевые слова, глубину, проф.область, город итд. Некоторые параметры, такие как проф.область="ит.телеком" и город="Москва" можно преднастроить по умолчанию).

К сожалению, поиск резюме в API отсутсвует, доступен лишь просмотр резюме через метод https://api.hh.ru/resumes/
Для поиска предлагается использовать подход с парсингом сайта и получением идентификаторов резюме с использованием библиотек Selenium и BeatifualSoap (на основе сформированного поиского запроса к web-версии сайта, например https://hh.ru/search/resume?text=JAVA&st=resumeSearch&logic=normal&pos=full_text&exp_period=all_time&exp_company_size=any&exp_industry=any&area=113&relocation=living_or_relocation&salary_from=&salary_to=&currency_code=RUR&experience=between1And3&education=higher&age_from=&age_to=&gender=unknown&employment=full&schedule=fullDay&language=eng.b1&order_by=relevance&search_period=30&items_on_page=100 ) .
Далее идентификаторы можно просматривать в цикле и формировать из них массив данных (предварительно авторизовавшись с использованием токена корпоративной учетной записи).


In [14]:
import pandas as pd
import pymorphy2, re
from nltk.tokenize import sent_tokenize
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from collections import Counter
import json

data = pd.read_json('data/data3.json')
data.head()

,title,href,last_work_plase
0,Менеджер проектов,/resume/c431ef0300038d8ce30039ed1f4e5761694130...,KFC
1,Специалист EXCEL,/resume/6493e03a0007ad971c0039ed1f413546517745...,Информационный центр Избирательной комиссии Ал...
2,Продюсер,/resume/7048e0450005e07e180039ed1f6b336d673276...,Cinémot Production
3,"CIO, Head of IT, IT manager / Head of IT Infra...",/resume/dfb43d360000af79440039ed1f736563726574...,“Cesar Satellite” JSC (security services)
4,комьюнити-менеджер,/resume/4e36bfe0000612d5b00039ed1f737574643832...,Wargaming.net / CООО Гейм Стрим


In [15]:
data["href"] = 'https://hh.ru'+data["href"]

In [16]:
data.head()

,title,href,last_work_plase
0,Менеджер проектов,https://hh.ru/resume/c431ef0300038d8ce30039ed1...,KFC
1,Специалист EXCEL,https://hh.ru/resume/6493e03a0007ad971c0039ed1...,Информационный центр Избирательной комиссии Ал...
2,Продюсер,https://hh.ru/resume/7048e0450005e07e180039ed1...,Cinémot Production
3,"CIO, Head of IT, IT manager / Head of IT Infra...",https://hh.ru/resume/dfb43d360000af79440039ed1...,“Cesar Satellite” JSC (security services)
4,комьюнити-менеджер,https://hh.ru/resume/4e36bfe0000612d5b00039ed1...,Wargaming.net / CООО Гейм Стрим


In [17]:
count_company = data["last_work_plase"].value_counts()
df_count_company = pd.DataFrame({'last_work_plase':count_company.index, 'count':count_company.values})
df_count_company.head(30)

,last_work_plase,count
0,Индивидуальное предпринимательство / частная п...,171
1,Сбербанк,53
2,Ростелеком,30
3,Яндекс,26
4,Фриланс,26
5,Мобильные ТелеСистемы (МТС),25
6,Яндекс.Практикум,18
7,Тинькофф Банк,17
8,Рокетбанк,15
9,X5 RETAIL GROUP,14


In [18]:
data_with_count = data.set_index('last_work_plase').join(df_count_company.set_index('last_work_plase'))
data_with_count.head()

,title,href,count
last_work_plase,,,
Омега,Мастер по ремонту компьютеров,https://hh.ru/resume/1ead0b9c0007e8cbb00039ed1...,1.0
FIX PRICE,Специалист,https://hh.ru/resume/2cda41cf0003caf37f0039ed1...,1.0
ИНТЕНШЕН,Системный администратор,https://hh.ru/resume/6e58864a000285dccc0039ed1...,1.0
Интернет-Магазины,Интернет-маркетолог,https://hh.ru/resume/4e5432c300072145b80039ed1...,1.0
ОО «ИЦ»,Системный администратор,https://hh.ru/resume/bc91f04400033060aa0039ed1...,1.0


In [19]:
sorted_data = data_with_count.sort_values(by=["count"], ascending=False)
sorted_data.to_csv('data_sort.csv', sep='\t', index=False)

In [20]:
sorted_data.head(10)

,title,href,count
last_work_plase,,,
Индивидуальное предпринимательство / частная практика / фриланс,"CPO, CTO / Продукт Менеджер / Руководитель про...",https://hh.ru/resume/08a4b9520007a241880039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,Специалист по контролю качества,https://hh.ru/resume/86624a51000376fa300039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,(Data Analyst) Специалист по управлению данными,https://hh.ru/resume/0ee8c4880007a897130039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,Интернет-маркетолог / маркетолог,https://hh.ru/resume/bf450d8b0005dd69a30039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,Frontend/Backend web-программист,https://hh.ru/resume/c933cb990001a6cfee0039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,Программист-разработчик C#,https://hh.ru/resume/ab0ae9060007deb5070039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,SEO-специалист,https://hh.ru/resume/ac0f67a3000803cf5f0039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,"Системный администратор (удаленка, приходящий,...",https://hh.ru/resume/dd94cb530007dd0ad00039ed1...,171.0
Индивидуальное предпринимательство / частная практика / фриланс,Project manager,https://hh.ru/resume/af09c01d00076db3890039ed1...,171.0


In [21]:
sorted_data.to_csv('sorted_data_java.csv', sep='\t', index=False)

In [25]:
sorted_data.drop(["Индивидуальное предпринимательство / частная практика / фриланс"], inplace = True)
sorted_data.drop(["Фриланс"], inplace = True)
sorted_data.drop(["Freelance"], inplace = True)
sorted_data.drop(["фриланс"], inplace = True)

In [26]:
sorted_data.head(107)

,title,href,count
last_work_plase,,,
Сбербанк,Администратор; Специалист; Аналитик,https://hh.ru/resume/a67382350002bf30e50039ed1...,53.0
Сбербанк,Главный инженер по разработке,https://hh.ru/resume/e14c73b9000386d6e30039ed1...,53.0
Сбербанк,Ведущий инженер по тестированию(удаленно),https://hh.ru/resume/e597d9790001fe61770039ed1...,53.0
Сбербанк,Специалист,https://hh.ru/resume/d47a51550002c590550039ed1...,53.0
Сбербанк,"Главный/ведущий инженер IT, Аналитик, Эксперт,...",https://hh.ru/resume/a6fd12ae0005b27f370039ed1...,53.0
...,...,...,...
Яндекс,Frontend-разработчик,https://hh.ru/resume/cab98f910007f54a490039ed1...,26.0
Яндекс,Системный администратор,https://hh.ru/resume/b84779390007c71d270039ed1...,26.0
Яндекс,Инженер,https://hh.ru/resume/e13945cc0000b9ef370039ed1...,26.0


In [119]:
sorted_data.to_csv('sorted_data_java.csv', sep='\t', index=False)